# A simple example on how to use `scopes`

In this tutorial you will learn the basic functionality of scopes and how to set everything up to use it.

## Topics

- Construct the "Night" object to designate the specific night of the year for which the schedule is to be created.
- Establish the observing programs, outlining their specific objectives and parameters.
- Determine the merits to be utilized, ensuring they align with the objectives of the observing programs.
- Create the "Target" objects, assigning appropriate merits based on the unique requirements of each target.
- Create the "Observation" objects, detailing the specifics of each observation.
- Compile a preliminary night schedule using the created observations, forming an organized plan for the designated night.

In [1]:
from scopes.scheduler_components import Night, Program, Merit, Target, Observation
from scopes import merits
from scopes import scheduler

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from datetime import date, timedelta
from astropy.coordinates import SkyCoord
import astroplan

### Observer and Night

We will start by defining the Observer, that is where in the world our telescope is located, and for which night we want to create a schedule. This is done using the Observer object from the astroplan package. For this example we will consider a telescope at the La Silla Observatory in Chile.

We then create the Night object for which night we want to create the schedule. This is done by defining the date, within which twilights observations should be considered

In [2]:
# Define observer location
observer = astroplan.Observer.at_site("lasilla")

# Define the night
night = Night(date(2023, 11, 14), "nautical", observer)

### Programs

SCOPES allows to define different observing programs. This is widely the case in many telescopes where different groups use the telescope for different purposes and thus each group or person has their own observing program with their own set of targets and scientific priorities.

These are defined with the Program object where the parameters to be given are the program ID, name of the isntrument to be used, the proprotion of the total time that is allocated to this program, and optionally with what color this program will be plotted.

In [3]:
# Define color pallette for plotting
color_pallette = iter([mcolors.rgb2hex(color) for color in plt.get_cmap("Set2").colors])

# Lets use "CAR" and "BIKE" as our two example instruments
prog1 = Program("prog1", "CAR", 0.1, 1, plot_color=next(color_pallette))
prog2 = Program("prog2", "CAR", 0.1, 2, plot_color=next(color_pallette))
prog3 = Program("prog3", "CAR", 0.3, 2, plot_color=next(color_pallette))
prog4 = Program("prog4", "BIKE", 0.2, 3, plot_color=next(color_pallette))
prog5 = Program("prog5", "BIKE", 0.3, 2, plot_color=next(color_pallette))

Now we will tell the scopes how much time each of these programs have taken up already. This is done by calling the `update_time_share()` method on each program and indicating the current time used by that program (as a percentage of the total time).

To simulate a somwwhat real situation, lets asume that prog1 and prog3 were observed less than was allocated to them, prog4 is close to even, while prog2 and prog5 were observed more than the time they were allocated.

In [ ]:
prog1.update_time_share(0.09)

## Merits

Now we will define the set of merits we will be using. Some standard merit functions can be found in the `scopes.merits` module, but custom ones can be defined as well.

The Merit class takes three mandatory arguments, `name`, `func`, and `merit_type`. `func` is the function that actually computes the merit from an observation, `merit_type` tells the scheduler how to use the merit in the rank function, i.e. if its a fariness, veto, or efficiency merit. Then the optional `paramteres` arguments gives the function any additional keyword arguments for that merit. This allows to create merits that use the same merit function but with a different set of parameters.

In [9]:
# We will start with the basic observability merits like limits on the airmass, altitude, and time of night.

# Lets define a merit for the airmass setting a maximum of 1.8
airmass_merit_18 = Merit(
    "Airmass", merits.airmass, merit_type="veto", parameters={"max": 1.8}
)

# Lets define a merit for the airmass setting a maximum of 1.5
airmass_merit_15 = Merit(
    "Airmass", merits.airmass, merit_type="veto", parameters={"max": 1.5}
)

# Lets define a merit for the altitude. These are usually set by the hardware limits of the telescope.
# In this exmaple we will use a minimum of 20 and a maximum of 87 degrees (to avoid the zenith
# which can cause tracking issues in telescopes with an altazimuth mount)
altitude_merit = Merit(
    "Altitude", merits.altitude, merit_type="veto", parameters={"min": 20, "max": 87}
)
# Lets define a merit for the time of night. These limits are used from the Night object we created above.
at_night_merit = Merit("AtNight", merits.at_night, merit_type="veto")

# The Culmination merit is used to ensure observations are done close to the culmination of the target in the sky.
culmapping_merit = Merit(
    "CulMapping", merits.culmination_mapping, merit_type="efficiency"
)

# Lastly, we will deifne a fairness merit for the time share. This merit ensures that programs
# respect the time share they are given.
timeshare_merit = Merit("Timeshare", merits.time_share, merit_type="fairness")
priority_merit = Merit("Priority", merits.priority, merit_type="fairness")

## Targets and Observations

Next we will define the targets to be observed by creating Target objects and then Observation Objects.

The Target object contain information about the target itself, like its name, coordinates, which program it is part of, its priority and most importantly its individual merit functions which determine how the target should be observed.

The observatio object takes a target as input and adds the information (like when it is observed and its exposure time) and logic to actually schedule the observation at an appropiate time. The reason the Target and Observation objects are distinct is to allow the posisblity to create more than one observation for the same target if needed.

**In this tutorial we will create simulated targets, but in this section is where one would actually load the actual targets that want to be observed.**

In [10]:
# Generate random star coordinates
np.random.seed(0)
ntars = 100
ra = np.random.uniform(0, 360, size=ntars)
dec = np.rad2deg(np.arccos(np.random.uniform(0, 1, size=ntars)))

# We will initialize the observations start time at the start of the night
start_time = night.night_time_range[0].jd


# Create Observations objects
def create_tars_and_obs(prog, idx_range, merits):
    observations = []
    for i in range(idx_range[0], idx_range[1]):
        # Get the coordinates for the target
        coord = SkyCoord(ra[i], dec[i], unit="deg")
        # Create the Target object with a random priority
        target = Target(f"Target{i}", prog, coord, np.random.randint(0, 4))
        # Add the merits to the target
        target.add_merits(merits)

        # Create the observation
        exposure_time = 900 / 86400  # 900 seconds in units of days
        observation = Observation(target, start_time, exposure_time, night)
        observations.append(observation)
    return observations

Now lets create the Observations for each program. Let's assume that prog1 and prog2 need their targets to be observed at a maximum airmass of 1.5 as they need a good SNR. In contrast, prog3, prog4, and prog5 are not that sensitive to SNR and allows their targets to be observed up to an airmass of 1.8.

In [11]:
merits1 = [
    airmass_merit_18,
    altitude_merit,
    at_night_merit,
    culmapping_merit,
    timeshare_merit,
    priority_merit,
]

merits2 = [
    airmass_merit_15,
    altitude_merit,
    at_night_merit,
    culmapping_merit,
    timeshare_merit,
    priority_merit,
]

obs_prog1 = create_tars_and_obs(prog1, (0, 20), merits1)
obs_prog2 = create_tars_and_obs(prog2, (20, 40), merits1)
obs_prog3 = create_tars_and_obs(prog3, (40, 60), merits2)
obs_prog4 = create_tars_and_obs(prog4, (60, 80), merits2)
obs_prog5 = create_tars_and_obs(prog5, (80, 100), merits2)

## Scheduling the night

We have set up everything we need to now create the night sschedule based on the observations we created.

To do this we will initialize a Scheduler object from the `scopes.scheudler` module. To start we will use the simple generateQ scheduler which works based on a simple greedy search algorithms which works sequentially from the beginning of the night and always chooses the best scoring observation as the next observation to do.

Target(Name: Target0,
       Program: prog1,
       Coordinates: (197.573, 47.327),
       Priority: 2,
       Fairness Merits: [Merit(Timeshare, fairness, {}), Merit(Priority, fairness, {})],
       Veto Merits: [Merit(Airmass, veto, {'max': 1.8}), Merit(Altitude, veto, {'min': 20, 'max': 87}), Merit(AtNight, veto, {})],
       Efficiency Merits: [Merit(CulMapping, efficiency, {})])